# Transport NSW Opal Card ML Pipeline - Complete Analysis

Portal: https://opendata.transport.nsw.gov.au

- Train Trips: https://opendata.transport.nsw.gov.au/dataset/opal-trips-train
  
    https://opendata.transport.nsw.gov.au/data/api/action/datastore_search?resource_id=7d49ca0b-1fb6-4524-81b0-68c8775b4d82

- Card type usage patterns: https://opendata.transport.nsw.gov.au/data/dataset/opal-trips-all-modes

    https://opendata.transport.nsw.gov.au/data/api/action/datastore_search?resource_id=c77b83d7-cdcd-4a0e-9b12-fcc159727589

**Business Questions Answered**:
1. Which train lines show declining ridership?
2. Predict next month's demand by route
3. Which stations need capacity upgrades?
4. Optimal train frequency per line
5. Card type usage patterns
6. Seasonal demand patterns

In [1]:
!pip install -q sagemaker scikit-learn pandas numpy matplotlib seaborn requests joblib

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")

✅ Libraries imported


## Step 1: Fetch Real Transport NSW Data

In [7]:
def fetch_train_data_with_lines():
    """
    Fetch Transport NSW data and properly extract train line names
    """
    print("🚇 Fetching REAL Transport NSW Train Data...\n")
    
    url = "https://opendata.transport.nsw.gov.au/data/api/action/datastore_search"
    params = {
        'resource_id': '7d49ca0b-1fb6-4524-81b0-68c8775b4d82',
        'limit': 1000000
    }
    
    try:
        response = requests.get(url, params=params, timeout=60)
        
        if response.status_code == 200:
            data = response.json()
            
            if 'result' in data and 'records' in data['result']:
                df = pd.DataFrame(data['result']['records'])
                
                print(f"✅ SUCCESS! Retrieved {len(df)} records")
                print(f"\n📊 Data Structure:")
                print(f"   Columns: {list(df.columns)}")
                print(f"\n   First few column names:")
                for col in df.columns[:10]:
                    print(f"      - {col}")
                
                return df, data['result'].get('fields', [])
            else:
                print("❌ Unexpected API response format")
                return None, None
        else:
            print(f"❌ API returned status: {response.status_code}")
            return None, None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None, None

df_raw, fields_info = fetch_train_data_with_lines()

if df_raw is not None:
    print(f"\n" + "="*80)
    print("📊 RAW DATA PREVIEW (First 3 rows)")
    print("="*80)
    print(df_raw.head(3))

🚇 Fetching REAL Transport NSW Train Data...

✅ SUCCESS! Retrieved 19820 records

📊 Data Structure:
   Columns: ['_id', 'Year_Month', 'Card_type', 'Line', 'Trip']

   First few column names:
      - _id
      - Year_Month
      - Card_type
      - Line
      - Trip

📊 RAW DATA PREVIEW (First 3 rows)
   _id Year_Month    Card_type                 Line    Trip
0    1     Jul-16        Adult  Blue Mountains Line  537669
1    2     Jul-16  Child/Youth  Blue Mountains Line   36387
2    3     Jul-16   Concession  Blue Mountains Line   60238


## Step 2: Identify Train Line Columns (FIXED)

In [8]:
if df_raw is not None:
    print("🔍 IDENTIFYING TRAIN LINE COLUMNS\n")
    print("="*80)
    
    # Look for columns that might contain line names
    print("\n1. ALL COLUMNS IN DATASET:")
    for i, col in enumerate(df_raw.columns, 1):
        # Check data type and sample values - FIXED: Convert dtype to string
        dtype = str(df_raw[col].dtype)
        sample = str(df_raw[col].iloc[0])[:50] if len(df_raw) > 0 else "N/A"
        print(f"   {i:2}. {col:40} | Type: {dtype:10} | Sample: {sample}")
    
    # Identify numeric columns (potential trip counts)
    numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
    print(f"\n2. NUMERIC COLUMNS (potential trip counts): {len(numeric_cols)}")
    for col in numeric_cols:
        print(f"   - {col}")
    
    # Look for line identifier columns
    potential_line_cols = [col for col in df_raw.columns if 
                          any(x in col.lower() for x in ['line', 'route', 'mode', 'service'])]
    print(f"\n3. POTENTIAL LINE IDENTIFIER COLUMNS: {len(potential_line_cols)}")
    for col in potential_line_cols:
        unique_vals = df_raw[col].nunique()
        print(f"   - {col:40} | {unique_vals} unique values")
        if unique_vals < 20:
            print(f"     Values: {df_raw[col].unique()[:10].tolist()}")
    
    # Check for date column
    date_cols = [col for col in df_raw.columns if any(x in col.lower() for x in ['date', 'month', 'year', 'period'])]
    print(f"\n4. DATE COLUMNS: {len(date_cols)}")
    for col in date_cols:
        print(f"   - {col}")
        print(f"     Sample values: {df_raw[col].head(3).tolist()}")
else:
    print("⚠️  No data to analyze")

🔍 IDENTIFYING TRAIN LINE COLUMNS


1. ALL COLUMNS IN DATASET:
    1. _id                                      | Type: int64      | Sample: 1
    2. Year_Month                               | Type: object     | Sample: Jul-16
    3. Card_type                                | Type: object     | Sample: Adult
    4. Line                                     | Type: object     | Sample: Blue Mountains Line
    5. Trip                                     | Type: int64      | Sample: 537669

2. NUMERIC COLUMNS (potential trip counts): 2
   - _id
   - Trip

3. POTENTIAL LINE IDENTIFIER COLUMNS: 1
   - Line                                     | 35 unique values

4. DATE COLUMNS: 1
   - Year_Month
     Sample values: ['Jul-16', 'Jul-16', 'Jul-16']


## Step 3: Reshape Data to Proper Format

In [9]:
def reshape_transport_data(df_raw):
    """
    Reshape the data so each train line is properly identified
    """
    if df_raw is None or len(df_raw) == 0:
        return None
    
    print("🔄 RESHAPING DATA FOR PROPER ANALYSIS\n")
    
    # Check if there's a column identifying the line/route
    line_identifier_cols = [col for col in df_raw.columns if 
                           any(x in col.lower() for x in ['line', 'route', 'mode'])]
    
    if line_identifier_cols:
        # Long format - already has line names
        print(f"✅ Data is in LONG format with line identifier: {line_identifier_cols[0]}")
        line_col = line_identifier_cols[0]
        
        # Find trip/count column
        trip_cols = [col for col in df_raw.columns if 
                    any(x in col.lower() for x in ['trip', 'count', 'patronage', 'boardings'])]
        
        if trip_cols:
            trip_col = trip_cols[0]
            print(f"✅ Found trip column: {trip_col}")
            
            # Find date column
            date_cols = [col for col in df_raw.columns if 
                        any(x in col.lower() for x in ['date', 'month', 'year', 'period'])]
            
            if date_cols:
                date_col = date_cols[0]
                print(f"✅ Found date column: {date_col}")
                
                # Create clean dataframe
                df_clean = df_raw[[date_col, line_col, trip_col]].copy()
                df_clean.columns = ['Date', 'Line', 'Trips']
                
                # Convert to numeric
                df_clean['Trips'] = pd.to_numeric(df_clean['Trips'], errors='coerce')
                df_clean = df_clean.dropna()
                
                # Parse date
                df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
                df_clean = df_clean.dropna(subset=['Date'])
                df_clean = df_clean.sort_values('Date')
                
                # Add time features
                df_clean['Year'] = df_clean['Date'].dt.year
                df_clean['Month'] = df_clean['Date'].dt.month
                df_clean['Quarter'] = df_clean['Date'].dt.quarter
                
                print(f"\n✅ Reshaped data: {len(df_clean)} records")
                print(f"   Unique lines: {df_clean['Line'].nunique()}")
                print(f"   Date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")
                print(f"\n📊 Train Lines Found:")
                for line in sorted(df_clean['Line'].unique()):
                    count = len(df_clean[df_clean['Line'] == line])
                    avg_trips = df_clean[df_clean['Line'] == line]['Trips'].mean()
                    print(f"      • {line:50} ({count:3} records, avg {avg_trips:>12,.0f} trips/month)")
                
                return df_clean
    
    # If wide format, need to melt it
    print("📊 Data appears to be in WIDE format (lines as columns)")
    print("   Attempting to identify train line columns...")
    
    # Find date column
    date_col = None
    for col in df_raw.columns:
        if any(x in col.lower() for x in ['year', 'month', 'date', 'period']):
            date_col = col
            break
    
    if date_col is None:
        print("   ⚠️  Could not identify date column")
        return None
    
    print(f"   ✅ Date column: {date_col}")
    
    # Assume all numeric columns except _id are train lines
    numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
    line_cols = [col for col in numeric_cols if col != '_id']
    
    if not line_cols:
        print("   ⚠️  Could not identify line columns")
        return None
    
    print(f"   ✅ Found {len(line_cols)} potential line columns")
    
    # Melt the dataframe
    df_melted = df_raw.melt(
        id_vars=[date_col],
        value_vars=line_cols,
        var_name='Line',
        value_name='Trips'
    )
    
    df_melted.columns = ['Date', 'Line', 'Trips']
    df_melted['Trips'] = pd.to_numeric(df_melted['Trips'], errors='coerce')
    df_melted = df_melted.dropna()
    
    df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')
    df_melted = df_melted.dropna(subset=['Date'])
    df_melted = df_melted.sort_values('Date')
    
    # Add time features
    df_melted['Year'] = df_melted['Date'].dt.year
    df_melted['Month'] = df_melted['Date'].dt.month
    df_melted['Quarter'] = df_melted['Date'].dt.quarter
    
    print(f"\n✅ Reshaped data: {len(df_melted)} records")
    print(f"   Unique lines: {df_melted['Line'].nunique()}")
    print(f"\n📊 Train Lines Found:")
    for line in sorted(df_melted['Line'].unique()):
        count = len(df_melted[df_melted['Line'] == line])
        avg_trips = df_melted[df_melted['Line'] == line]['Trips'].mean()
        print(f"      • {line:50} ({count:3} records, avg {avg_trips:>12,.0f} trips/month)")
    
    return df_melted

df_clean = reshape_transport_data(df_raw)

if df_clean is not None:
    print("\n" + "="*80)
    print("✅ DATA READY FOR ANALYSIS")
    print("="*80)
    print(df_clean.head(15))

🔄 RESHAPING DATA FOR PROPER ANALYSIS

✅ Data is in LONG format with line identifier: Line
✅ Found trip column: Trip
✅ Found date column: Year_Month

✅ Reshaped data: 5070 records
   Unique lines: 25
   Date range: 2024-01-01 00:00:00 to 2025-09-01 00:00:00

📊 Train Lines Found:
      • Airport replacement buses                          ( 69 records, avg          283 trips/month)
      • Bankstown Replacement Buses                        ( 29 records, avg           11 trips/month)
      • Blue Mountains Line                                (266 records, avg       43,418 trips/month)
      • Carlingford Replacement Buses                      (111 records, avg        2,393 trips/month)
      • Central Coast & Newcastle Line                     (267 records, avg       82,429 trips/month)
      • Hunter Line                                        (234 records, avg        5,891 trips/month)
      • NSW TrainLink Southern Train Services              (141 records, avg            0 trips/month)


## Business Question 1: Which train lines show declining ridership?

In [10]:
if df_clean is not None and len(df_clean) > 0:
    print("="*80)
    print("❓ QUESTION 1: WHICH TRAIN LINES SHOW DECLINING RIDERSHIP?")
    print("="*80)
    
    lines = df_clean['Line'].unique()
    print(f"\n📊 Analyzing {len(lines)} train lines\n")
    
    trends = {}
    
    for line in lines:
        line_data = df_clean[df_clean['Line'] == line].sort_values('Date')
        
        if len(line_data) > 3:
            x = np.arange(len(line_data))
            y = line_data['Trips'].values
            z = np.polyfit(x, y, 1)
            slope = z[0]
            first_val = y[0] if y[0] != 0 else 1
            pct_change = (slope * len(line_data) / first_val) * 100
            
            trends[line] = {
                'slope': slope,
                'pct_change': pct_change,
                'current': y[-1],
                'avg': y.mean(),
                'records': len(line_data)
            }
    
    sorted_trends = sorted(trends.items(), key=lambda x: x[1]['pct_change'])
    
    print("📉 DECLINING LINES (Negative Growth):\n")
    declining_count = 0
    for line, data in sorted_trends:
        if data['pct_change'] < -1:
            declining_count += 1
            print(f"   {line:50} {data['pct_change']:+7.1f}% | Avg: {data['avg']:>10,.0f} trips/month")
            if declining_count >= 10:
                break
    
    if declining_count == 0:
        print("   ✅ No lines showing significant decline")
    
    print("\n📈 FASTEST GROWING LINES (Positive Growth):\n")
    growing_count = 0
    for line, data in reversed(sorted_trends):
        if data['pct_change'] > 1:
            growing_count += 1
            print(f"   {line:50} {data['pct_change']:+7.1f}% | Avg: {data['avg']:>10,.0f} trips/month")
            if growing_count >= 10:
                break
    
    if growing_count == 0:
        print("   ⚠️  No lines showing significant growth")
    
    print("\n💼 BUSINESS RECOMMENDATIONS:\n")
    
    if declining_count > 0:
        worst = sorted_trends[0]
        print(f"   🚨 URGENT: {worst[0]}")
        print(f"      - Decline: {worst[1]['pct_change']:.1f}%")
        print(f"      - Current: {worst[1]['current']:,.0f} trips/month")
        print(f"      - Actions: Survey passengers, check competitors, review service quality")
    
    if growing_count > 0:
        best = sorted_trends[-1]
        print(f"\n   ✅ SUCCESS: {best[0]}")
        print(f"      - Growth: {best[1]['pct_change']:.1f}%")
        print(f"      - Current: {best[1]['current']:,.0f} trips/month")
        print(f"      - Actions: Monitor capacity, consider service expansion")
else:
    print("⚠️  No data available")

❓ QUESTION 1: WHICH TRAIN LINES SHOW DECLINING RIDERSHIP?

📊 Analyzing 25 train lines

📉 DECLINING LINES (Negative Growth):

   T1 North Shore Line & T1 Western Line              -4148263.3% | Avg:    540,184 trips/month
   T4 Eastern Suburbs & Illawarra Line                -1214257.4% | Avg:    410,646 trips/month
   Central Coast & Newcastle Line                     -1163927.0% | Avg:     82,429 trips/month
   Blue Mountains Line                                -481387.5% | Avg:     43,418 trips/month
   T5 Cumberland Line                                 -455185.9% | Avg:     50,596 trips/month
   T3 Bankstown Line & T3 Liverpool & Inner West Line -65118.8% | Avg:     84,191 trips/month
   Southern NSW                                        -332.9% | Avg:        172 trips/month
   North Coast NSW                                     -193.4% | Avg:        128 trips/month
   Carlingford Replacement Buses                       -123.9% | Avg:      2,393 trips/month
   Western NSW          

## Business Question 2: Predict next month's demand by route

In [11]:
if df_clean is not None and len(df_clean) > 20:
    print("="*80)
    print("❓ QUESTION 2: PREDICT NEXT MONTH'S DEMAND BY ROUTE")
    print("="*80)
    
    line_counts = df_clean.groupby('Line').size()
    target_line = line_counts.idxmax()
    
    print(f"\n🎯 Training model for: {target_line}")
    
    line_data = df_clean[df_clean['Line'] == target_line].sort_values('Date').copy()
    line_data['Lag_1'] = line_data['Trips'].shift(1)
    line_data['Lag_3'] = line_data['Trips'].shift(3)
    line_data['Rolling_3'] = line_data['Trips'].rolling(window=3).mean()
    line_data = line_data.dropna()
    
    print(f"   Training samples: {len(line_data)}")
    
    if len(line_data) > 10:
        features = ['Year', 'Month', 'Quarter', 'Lag_1', 'Lag_3', 'Rolling_3']
        X = line_data[features]
        y = line_data['Trips']
        
        split_idx = int(len(line_data) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        print(f"   Train: {len(X_train)}, Test: {len(X_test)}")
        
        print("\n🤖 Training Random Forest...")
        model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        
        print("\n📊 MODEL PERFORMANCE:\n")
        print(f"   Line: {target_line}")
        print(f"   RMSE: {rmse:,.0f} trips")
        print(f"   MAE:  {mae:,.0f} trips")
        print(f"   R²:   {r2:.4f}")
        print(f"   MAPE: {mape:.2f}%")
        
        print("\n🔮 PREDICTIONS (Last 5 test periods):\n")
        for i in range(min(5, len(y_test))):
            idx = -5 + i
            actual = y_test.values[idx]
            predicted = y_pred[idx]
            error_pct = ((actual - predicted) / actual) * 100
            print(f"   Actual: {actual:>10,.0f} | Predicted: {predicted:>10,.0f} | Error: {error_pct:+6.1f}%")
        
    else:
        print("   ⚠️  Insufficient data for this line")
else:
    print("⚠️  Not enough data for ML training")

❓ QUESTION 2: PREDICT NEXT MONTH'S DEMAND BY ROUTE

🎯 Training model for: T1 North Shore Line & T1 Western Line
   Training samples: 270
   Train: 216, Test: 54

🤖 Training Random Forest...

📊 MODEL PERFORMANCE:

   Line: T1 North Shore Line & T1 Western Line
   RMSE: 734,777 trips
   MAE:  448,042 trips
   R²:   0.2518
   MAPE: inf%

🔮 PREDICTIONS (Last 5 test periods):

   Actual:    575,922 | Predicted:    273,350 | Error:  +52.5%
   Actual:    257,622 | Predicted:    521,206 | Error: -102.3%
   Actual:        228 | Predicted:    274,455 | Error: -120274.9%
   Actual:      9,130 | Predicted:    213,687 | Error: -2240.5%
   Actual:     48,061 | Predicted:     21,056 | Error:  +56.2%


## Business Question 3: Which stations need capacity upgrades?

In [12]:
if df_clean is not None and len(df_clean) > 0:
    print("="*80)
    print("❓ QUESTION 3: WHICH STATIONS/LINES NEED CAPACITY UPGRADES?")
    print("="*80)
    
    # Calculate recent average (last 6 months)
    recent_data = df_clean.sort_values('Date').groupby('Line').tail(6)
    
    capacity_analysis = []
    CAPACITY_THRESHOLD = 1_500_000  # Assume 1.5M trips/month as capacity
    
    for line in df_clean['Line'].unique():
        line_recent = recent_data[recent_data['Line'] == line]
        
        if len(line_recent) > 0:
            avg_trips = line_recent['Trips'].mean()
            max_trips = line_recent['Trips'].max()
            capacity_used_pct = (avg_trips / CAPACITY_THRESHOLD) * 100
            
            capacity_analysis.append({
                'Line': line,
                'Avg_Monthly': avg_trips,
                'Max_Monthly': max_trips,
                'Capacity_Used': capacity_used_pct
            })
    
    capacity_df = pd.DataFrame(capacity_analysis).sort_values('Capacity_Used', ascending=False)
    
    print(f"\n⚠️  CAPACITY ANALYSIS (Threshold: {CAPACITY_THRESHOLD:,} trips/month)\n")
    print("Status: 🔴 >90% | 🟡 75-90% | 🟢 <75%\n")
    
    critical_count = 0
    warning_count = 0
    
    for idx, row in capacity_df.head(15).iterrows():
        if row['Capacity_Used'] > 90:
            status = "🔴 CRITICAL"
            critical_count += 1
        elif row['Capacity_Used'] > 75:
            status = "🟡 WARNING"
            warning_count += 1
        else:
            status = "🟢 OK"
        
        print(f"{status:12} {row['Line'][:45]:45} {row['Capacity_Used']:6.1f}% | Avg: {row['Avg_Monthly']:>10,.0f}")
    
    print("\n💼 INVESTMENT RECOMMENDATIONS:\n")
    
    if critical_count > 0:
        total_investment = critical_count * 50_000_000
        print(f"   🚨 URGENT: {critical_count} lines need immediate capacity expansion")
        print(f"   💰 Estimated investment: ${total_investment:,}")
        print(f"   📊 Actions:")
        print(f"      - Add rolling stock (trains)")
        print(f"      - Increase service frequency")
        print(f"      - Upgrade signaling for closer headways")
    
    if warning_count > 0:
        print(f"\n   ⚠️  MONITOR: {warning_count} lines approaching capacity")
        print(f"   📊 Plan upgrades for next 2-3 years")
    
    if critical_count == 0 and warning_count == 0:
        print("   ✅ All lines operating within safe capacity limits")
else:
    print("⚠️  No data available")

❓ QUESTION 3: WHICH STATIONS/LINES NEED CAPACITY UPGRADES?

⚠️  CAPACITY ANALYSIS (Threshold: 1,500,000 trips/month)

Status: 🔴 >90% | 🟡 75-90% | 🟢 <75%

🟢 OK         T2 Inner West & Leppington Line                 23.0% | Avg:    345,454
🟢 OK         T1 North Shore Line & T1 Western Line           10.0% | Avg:    149,799
🟢 OK         T9 Northern Line                                 5.9% | Avg:     88,155
🟢 OK         T8 Airport & South Line                          5.9% | Avg:     88,124
🟢 OK         T3 Bankstown Line                                4.0% | Avg:     59,520
🟢 OK         Central Coast & Newcastle Line                   2.3% | Avg:     34,364
🟢 OK         T4 Eastern Suburbs & Illawarra Line              2.2% | Avg:     32,700
🟢 OK         T2 Inner West & Leppington Line & T2 Leppingt    2.1% | Avg:     31,809
🟢 OK         T3 Bankstown Line & T3 Liverpool & Inner West    1.4% | Avg:     21,719
🟢 OK         South Coast Line                                 1.1% | Avg:     15,

## Business Question 4: Optimal train frequency per line

In [13]:
if df_clean is not None and len(df_clean) > 0:
    print("="*80)
    print("❓ QUESTION 4: OPTIMAL TRAIN FREQUENCY PER LINE")
    print("="*80)
    
    recent_data = df_clean.sort_values('Date').groupby('Line').tail(6)
    
    print("\n🚂 FREQUENCY RECOMMENDATIONS (Based on last 6 months):\n")
    print("Formula: Daily trips ÷ Train capacity (1,000 passengers) ÷ Operating hours\n")
    
    frequency_recommendations = []
    
    for line in df_clean['Line'].unique():
        line_recent = recent_data[recent_data['Line'] == line]
        
        if len(line_recent) > 0:
            avg_monthly = line_recent['Trips'].mean()
            avg_daily = avg_monthly / 30
            trains_needed_daily = avg_daily / 1000  # Capacity = 1000 passengers/train
            operating_hours = 19  # 5am-12am
            trains_per_hour = trains_needed_daily / operating_hours
            minutes_between = 60 / trains_per_hour if trains_per_hour > 0 else 60
            
            frequency_recommendations.append({
                'Line': line,
                'Daily_Trips': avg_daily,
                'Trains_Per_Hour': trains_per_hour,
                'Minutes_Between': minutes_between
            })
    
    freq_df = pd.DataFrame(frequency_recommendations).sort_values('Trains_Per_Hour', ascending=False)
    
    for idx, row in freq_df.head(15).iterrows():
        if row['Minutes_Between'] <= 5:
            rec = "Every 3-5 min (Metro frequency)"
        elif row['Minutes_Between'] <= 10:
            rec = "Every 6-10 min (High frequency)"
        elif row['Minutes_Between'] <= 20:
            rec = "Every 15-20 min (Standard)"
        else:
            rec = "Every 30+ min (Low frequency)"
        
        print(f"  {row['Line'][:45]:45} | {row['Trains_Per_Hour']:5.1f}/hr | {rec}")
    
    print("\n💼 OPERATIONAL RECOMMENDATIONS:\n")
    
    high_freq = freq_df[freq_df['Minutes_Between'] <= 10]
    if len(high_freq) > 0:
        print(f"   🚄 {len(high_freq)} lines require high-frequency service (≤10 min)")
        print(f"   📊 Actions: Automated trains, dedicated staff, priority maintenance")
    
    low_freq = freq_df[freq_df['Minutes_Between'] > 30]
    if len(low_freq) > 0:
        print(f"\n   🚃 {len(low_freq)} lines can operate with lower frequency (>30 min)")
        print(f"   💰 Potential savings: ${len(low_freq) * 2_000_000:,}/year")
else:
    print("⚠️  No data available")

❓ QUESTION 4: OPTIMAL TRAIN FREQUENCY PER LINE

🚂 FREQUENCY RECOMMENDATIONS (Based on last 6 months):

Formula: Daily trips ÷ Train capacity (1,000 passengers) ÷ Operating hours

  T2 Inner West & Leppington Line               |   0.6/hr | Every 30+ min (Low frequency)
  T1 North Shore Line & T1 Western Line         |   0.3/hr | Every 30+ min (Low frequency)
  T9 Northern Line                              |   0.2/hr | Every 30+ min (Low frequency)
  T8 Airport & South Line                       |   0.2/hr | Every 30+ min (Low frequency)
  T3 Bankstown Line                             |   0.1/hr | Every 30+ min (Low frequency)
  Central Coast & Newcastle Line                |   0.1/hr | Every 30+ min (Low frequency)
  T4 Eastern Suburbs & Illawarra Line           |   0.1/hr | Every 30+ min (Low frequency)
  T2 Inner West & Leppington Line & T2 Leppingt |   0.1/hr | Every 30+ min (Low frequency)
  T3 Bankstown Line & T3 Liverpool & Inner West |   0.0/hr | Every 30+ min (Low frequency)
  

## Business Question 5: Card type usage patterns

**Using dedicated card type API endpoint**

In [14]:
def fetch_card_type_data():
    """
    Fetch card type usage data from dedicated endpoint
    """
    print("="*80)
    print("❓ QUESTION 5: CARD TYPE USAGE PATTERNS")
    print("="*80)
    print("\n🎫 Fetching card type data from dedicated API...\n")
    
    url = "https://opendata.transport.nsw.gov.au/data/api/action/datastore_search"
    params = {
        'resource_id': 'c77b83d7-cdcd-4a0e-9b12-fcc159727589',
        'limit': 100000
    }
    
    try:
        response = requests.get(url, params=params, timeout=60)
        
        if response.status_code == 200:
            data = response.json()
            
            if 'result' in data and 'records' in data['result']:
                df_cards = pd.DataFrame(data['result']['records'])
                
                print(f"✅ SUCCESS! Retrieved {len(df_cards)} records")
                print(f"   Columns: {list(df_cards.columns)}\n")
                
                return df_cards
            else:
                print("❌ Unexpected API response format")
                return None
        else:
            print(f"❌ API returned status: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

df_cards = fetch_card_type_data()

if df_cards is not None and len(df_cards) > 0:
    # Explore the data structure
    print("📊 CARD TYPE DATA STRUCTURE:\n")
    print("Sample record:")
    print(df_cards.head(3))
    print("\n" + "="*80)
    
    # Identify card type columns
    card_type_keywords = ['Adult', 'Child', 'Senior', 'Concession', 'School', 'Student']
    card_cols = [col for col in df_cards.columns if any(card.lower() in col.lower() for card in card_type_keywords)]
    
    if len(card_cols) > 0:
        print(f"\n🎫 Found {len(card_cols)} card type columns:\n")
        for col in card_cols:
            print(f"   • {col}")
        
        # Calculate totals
        card_totals = {}
        for col in card_cols:
            try:
                # Convert to numeric and sum
                total = pd.to_numeric(df_cards[col], errors='coerce').sum()
                if total > 0:
                    card_totals[col] = total
            except Exception as e:
                print(f"   ⚠️  Could not process {col}: {str(e)}")
        
        if card_totals:
            total_all = sum(card_totals.values())
            
            print("\n📊 CARD TYPE DISTRIBUTION:\n")
            sorted_cards = sorted(card_totals.items(), key=lambda x: x[1], reverse=True)
            
            for card, count in sorted_cards:
                pct = (count / total_all) * 100
                bar = '█' * int(pct / 2)
                # Clean up card name
                card_name = card.replace('_', ' ').title()
                print(f"  {card_name[:35]:35} {count:>15,.0f} trips ({pct:5.1f}%) {bar}")
            
            print(f"\n  {'TOTAL':35} {total_all:>15,.0f} trips (100.0%)")
            
            # Business insights
            print("\n💼 BUSINESS INSIGHTS:\n")
            
            top_segment = sorted_cards[0]
            top_name = top_segment[0].replace('_', ' ').title()
            top_pct = (top_segment[1] / total_all) * 100
            print(f"   💳 PRIMARY SEGMENT: {top_name}")
            print(f"      - {top_pct:.1f}% of all trips")
            print(f"      - {top_segment[1]:,.0f} total trips")
            
            # Identify concession cards
            concession_keywords = ['concession', 'senior', 'pension']
            concession_cards = [c for c in sorted_cards if any(kw in c[0].lower() for kw in concession_keywords)]
            
            if concession_cards:
                conc_total = sum(c[1] for c in concession_cards)
                conc_pct = (conc_total / total_all) * 100
                print(f"\n   👴 CONCESSION/SENIOR SEGMENT:")
                print(f"      - {conc_pct:.1f}% of trips")
                print(f"      - {conc_total:,.0f} total trips")
                if conc_pct < 20:
                    print(f"      - 💡 GROWTH OPPORTUNITY: Increase seniors engagement")
                    print(f"      - Actions: Better accessibility, off-peak discounts")
            
            # Adult revenue analysis
            adult_cards = [c for c in sorted_cards if 'adult' in c[0].lower()]
            if adult_cards:
                adult_total = sum(c[1] for c in adult_cards)
                adult_pct = (adult_total / total_all) * 100
                
                if adult_pct > 50:
                    print(f"\n   💰 REVENUE STRATEGY:")
                    print(f"      - Adult cards: {adult_pct:.1f}% of trips")
                    print(f"      - Primary revenue driver")
                    print(f"      - Focus areas:")
                    print(f"        • Peak-hour service reliability")
                    print(f"        • Premium express services")
                    print(f"        • Workplace partnership programs")
            
            # Child/Student analysis
            youth_keywords = ['child', 'student', 'school']
            youth_cards = [c for c in sorted_cards if any(kw in c[0].lower() for kw in youth_keywords)]
            
            if youth_cards:
                youth_total = sum(c[1] for c in youth_cards)
                youth_pct = (youth_total / total_all) * 100
                print(f"\n   🎒 CHILD/STUDENT SEGMENT:")
                print(f"      - {youth_pct:.1f}% of trips")
                print(f"      - Peak during school terms")
                print(f"      - Actions: School holiday service adjustments")
            
            # ROI calculation
            print("\n💵 REVENUE OPTIMIZATION POTENTIAL:\n")
            
            # If adults are >60%, optimize for them
            if adult_pct > 60:
                print(f"   📊 SCENARIO: +5% Adult retention through improved service")
                print(f"      Current adult trips: {adult_total:,.0f}")
                print(f"      Additional trips: {adult_total * 0.05:,.0f}")
                print(f"      Revenue increase: ${adult_total * 0.05 * 3.5:,.0f} (@$3.50/trip)")
            
            # If concessions are <20%, there's growth opportunity
            if concession_cards and conc_pct < 20:
                print(f"\n   📊 SCENARIO: +3% Concession growth through accessibility")
                print(f"      Current concession trips: {conc_total:,.0f}")
                print(f"      Additional trips: {conc_total * 0.03:,.0f}")
                print(f"      Revenue increase: ${conc_total * 0.03 * 1.75:,.0f} (@$1.75/trip)")
            
        else:
            print("   ⚠️  Could not calculate card type totals")
    else:
        print("\n   ⚠️  No card type columns found in this dataset")
        print("\n   📊 Data Structure:")
        for col in df_cards.columns:
            print(f"      • {col}: {str(df_cards[col].dtype)}")
else:
    print("\n⚠️  Could not fetch card type data")
    print("   Check: Internet connection, API availability")

❓ QUESTION 5: CARD TYPE USAGE PATTERNS

🎫 Fetching card type data from dedicated API...

✅ SUCCESS! Retrieved 6342 records
   Columns: ['_id', 'Year_Month', 'Card_type', 'Travel_Mode', 'Trip']

📊 CARD TYPE DATA STRUCTURE:

Sample record:
   _id Year_Month    Card_type Travel_Mode        Trip
0    1   Jul-2016        Adult         Bus  13146432.0
1    2   Jul-2016  Child/Youth         Bus   1079640.0
2    3   Jul-2016   Concession         Bus   1845322.0


   ⚠️  No card type columns found in this dataset

   📊 Data Structure:
      • _id: int64
      • Year_Month: object
      • Card_type: object
      • Travel_Mode: object
      • Trip: float64


## Business Question 6: Seasonal Demand Patterns

In [15]:
if df_clean is not None and len(df_clean) > 12:
    print("="*80)
    print("❓ QUESTION 6: SEASONAL DEMAND PATTERNS")
    print("="*80)
    
    if 'Month' in df_clean.columns:
        monthly_avg = df_clean.groupby('Month')['Trips'].mean().sort_index()
        
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        seasonal_data = pd.DataFrame({
            'Month': [month_names[i-1] for i in monthly_avg.index],
            'Avg_Trips': monthly_avg.values
        })
        
        peak_month = seasonal_data.loc[seasonal_data['Avg_Trips'].idxmax()]
        low_month = seasonal_data.loc[seasonal_data['Avg_Trips'].idxmin()]
        
        print("\n📅 MONTHLY DEMAND PATTERN:\n")
        
        for idx, row in seasonal_data.iterrows():
            pct_of_peak = (row['Avg_Trips'] / peak_month['Avg_Trips']) * 100
            bar = '█' * int(pct_of_peak / 5)
            
            if row['Month'] == peak_month['Month']:
                marker = '🔴 PEAK'
            elif row['Month'] == low_month['Month']:
                marker = '🔵 LOW'
            else:
                marker = '  '
            
            print(f"  {marker:8} {row['Month']:4} {row['Avg_Trips']:>12,.0f} trips {bar}")
        
        variation_pct = ((peak_month['Avg_Trips'] - low_month['Avg_Trips']) / low_month['Avg_Trips']) * 100
        
        print(f"\n📊 SEASONALITY METRICS:\n")
        print(f"   Peak Month: {peak_month['Month']} - {peak_month['Avg_Trips']:,.0f} trips")
        print(f"   Low Month:  {low_month['Month']} - {low_month['Avg_Trips']:,.0f} trips")
        print(f"   Variation:  {variation_pct:.1f}%")
        
        print("\n💼 OPERATIONAL RECOMMENDATIONS:\n")
        
        print(f"   🚂 PEAK PERIOD ({peak_month['Month']})")
        print(f"      - Increase services by {variation_pct/2:.0f}%")
        print(f"      - All staff on duty")
        print(f"      - Defer non-critical maintenance")
        
        print(f"\n   🔧 LOW PERIOD ({low_month['Month']})")
        print(f"      - Reduce services by {variation_pct/3:.0f}%")
        print(f"      - Schedule major maintenance")
        print(f"      - Staff training and leave")
        print(f"      - Cost savings: ${variation_pct * 500000:,.0f}/month")
        
        summer_months = [12, 1, 2]
        winter_months = [6, 7, 8]
        
        summer_avg = seasonal_data[seasonal_data.index.isin([i-1 for i in summer_months if i <= 12])]['Avg_Trips'].mean()
        winter_avg = seasonal_data[seasonal_data.index.isin([i-1 for i in winter_months])]['Avg_Trips'].mean()
        
        print(f"\n   🌞 Summer average: {summer_avg:,.0f} trips/month")
        print(f"   ❄️  Winter average: {winter_avg:,.0f} trips/month")
        
        if summer_avg < winter_avg:
            print(f"   📊 School holidays reduce patronage in summer")
        else:
            print(f"   📊 Tourist activity increases summer patronage")
    else:
        print("   ⚠️  Month column not available")
else:
    print("⚠️  Need at least 12 months of data")

❓ QUESTION 6: SEASONAL DEMAND PATTERNS

📅 MONTHLY DEMAND PATTERN:

           Jan       106,687 trips ███████████████
           Feb       120,458 trips █████████████████
  🔴 PEAK   Mar       138,013 trips ████████████████████
           Apr       119,660 trips █████████████████
           May       131,684 trips ███████████████████
           Jun       118,812 trips █████████████████
           Jul       125,205 trips ██████████████████
           Aug       123,521 trips █████████████████
           Sep       132,572 trips ███████████████████
           Oct       125,065 trips ██████████████████
           Nov       113,873 trips ████████████████
  🔵 LOW    Dec       105,195 trips ███████████████

📊 SEASONALITY METRICS:

   Peak Month: Mar - 138,013 trips
   Low Month:  Dec - 105,195 trips
   Variation:  31.2%

💼 OPERATIONAL RECOMMENDATIONS:

   🚂 PEAK PERIOD (Mar)
      - Increase services by 16%
      - All staff on duty
      - Defer non-critical maintenance

   🔧 LOW PERIOD (Dec)
